In [ ]:
from PPSHitmaps import PPSHitmap

# Describe the hitmaps

hitmap = {
    "map196-calib.out": PPSHitmap("MarioHitmaps/map196-calib.out", "196", 10, physics=False, calib=True),
    "map196-physics.out": PPSHitmap("MarioHitmaps/map196-physics.out", "196", 10, verbose=True),#, minFlux=10**6),
    "map220-calib.out": PPSHitmap("MarioHitmaps/map220-calib.out", "220", 4.25, physics=False, calib=True),
    "map220-physics.out": PPSHitmap("MarioHitmaps/map220-physics.out", "220", 4.25, verbose=True),#, minFlux=10**6),
    "map234-calib.out": PPSHitmap("MarioHitmaps/map234-calib.out", "234", 1.25, yStep=0.000025, xStep=0.000025, physics=False, calib=True),
    "map234-physics.out": PPSHitmap("MarioHitmaps/map234-physics.out", "234", 1.25, yStep=0.000025, xStep=0.000025),#, minFlux=10**6),
    "map420-calib.out": PPSHitmap("MarioHitmaps/map420-calib.out", "420", 6.6, yMin=-0.025, yMax=0.025, xMax=0.025, physics=False, calib=True),
    "map420-physics.out": PPSHitmap("MarioHitmaps/map420-physics.out", "420", 6.6, yMin=-0.025, yMax=0.025, xMax=0.025),#, minFlux=10**6),
}

print("Validating the hitmap files:")
for file in hitmap:
    print("  - {}".format(file))
    hitmap[file].validate()



In [ ]:
# Generate hitmap histograms and occupancy graphs

maxPadSize = 1.0E-2

hist_phys = {}
hist_calib = {}
occupancyGraph = {}
occupancyGraphCalib = {}
for station in ["196", "220", "234", "420"]:
    mapStr = "map{}-physics.out".format(station)
    if mapStr in hitmap:
        minPadSize = hitmap[mapStr].xStep
        padSizeBins = int(maxPadSize/minPadSize * 0.5)
        thisMaxPadSize = padSizeBins * minPadSize * 2

        hist_phys[station]  = hitmap[mapStr].getHisto("station{}-physics".format(station), "Station {}".format(station))

        occupancyGraph[station] = {}
        occupancyGraph[station]["uniform"]   = hitmap[mapStr].squarePadPeakUniformGraph(50, 1e-6, 1e-2, padScale = 1.0E6, doLog = True)
        occupancyGraph[station]["integrate"] = hitmap[mapStr].squarePadIntegrateGraph(50, 1e-6, 1e-2, padScale = 1.0E6, doLog = True)
        occupancyGraph[station]["points"]    = hitmap[mapStr].squarePadIntegrateGraph(padSizeBins, minPadSize, thisMaxPadSize, padScale = 1.0E6)
        hitmap[mapStr]._freeMap()


    mapStr = "map{}-calib.out".format(station)
    if mapStr in hitmap:
        minPadSize = hitmap[mapStr].xStep
        padSizeBins = int(maxPadSize/minPadSize * 0.5)
        thisMaxPadSize = padSizeBins * minPadSize * 2

        hist_calib[station] = hitmap[mapStr].getHisto("station{}-calib".format(station),   "Station {}".format(station))

        occupancyGraphCalib[station] = {}
        occupancyGraphCalib[station]["uniform"]   = hitmap[mapStr].squarePadPeakUniformGraph(50, 1e-6, 1e-2, padScale = 1.0E6, doLog = True)
        occupancyGraphCalib[station]["integrate"] = hitmap[mapStr].squarePadIntegrateGraph(50, 1e-6, 1e-2, padScale = 1.0E6, doLog = True)
        occupancyGraphCalib[station]["points"]    = hitmap[mapStr].squarePadIntegrateGraph(padSizeBins, minPadSize, thisMaxPadSize, padScale = 1.0E6)
        hitmap[mapStr]._freeMap()

In [ ]:
from ROOT import TCanvas
from ROOT import TLine
from ROOT import kRed

canv = TCanvas("physics-detectorEdge", "physics-detectorEdge", 1200, 1200)
canv.Divide(2,2)

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    mapStr = "map{}-physics.out".format(station)
    if mapStr not in hitmap:
        continue

    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    #hist_phys[station].SetMinimum(1e6)
    #hist_phys[station].SetMaximum(1e16)
    
    hist_phys[station].Draw("colz")
    
    edge = hitmap["map{}-physics.out".format(station)].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmap["map{}-physics.out".format(station)].yMin * 1000         # Convert to mm for drawing
    yMax = hitmap["map{}-physics.out".format(station)].yMax * 1000         # Convert to mm for drawing
    line[station] = TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(kRed)
    line[station].Draw("same")
canv.Draw()

In [ ]:
from ROOT import TCanvas, TH2D, kRed

canv = TCanvas("occupancy_vs_padSize", "Occupancy vs Pad Size", 1200, 1200)
canv.Divide(2,2)

# Create an empty histogram to serve as the frame for the graphs
frame = TH2D("frame", "", 100, 10, 10000, 100, 1.0E-5, 1)
frame.SetStats(False)
frame.GetXaxis().SetTitle("Pad Size [#mum]")
frame.GetYaxis().SetTitle("#mu")

idx = 0
for station in ["196", "220", "234", "420"]:
    mapStr = "map{}-physics.out".format(station)
    if mapStr not in hitmap:
        continue

    idx += 1
    pad = canv.cd(idx)
    pad.SetLogx()
    pad.SetLogy()
    pad.SetTicks()

    occupancyGraph[station]["frame"] = frame.Clone("frame-{}".format(station))
    occupancyGraph[station]["frame"].SetTitle("Station {}".format(station))
    occupancyGraph[station]["frame"].Draw()

    occupancyGraph[station]["uniform"].Draw("l same")
    occupancyGraph[station]["integrate"].Draw("l same")
    occupancyGraph[station]["points"].SetMarkerStyle(20)
    occupancyGraph[station]["points"].Draw("p same")

canv.Draw()

In [ ]:
from ROOT import TGraph
from array import array
import ctypes
from math import exp

def calcLossProb(timeStep, occupancy):
    return 1 - (occupancy ** 2)/((1 - exp(-occupancy))**2) * exp(-2*occupancy * (timeStep + 1))

def lossProbDistrib(timeStep, occupancy):
    lossProb = []
    
    for val in occupancy:
        lossProb += [calcLossProb(timeStep, val)]
        
    return lossProb

allTimeSteps = range(400)
drawTimeSteps = [0, 1, 3, 7, 39, 399]

lossProbability = {}
for station in ["196", "220", "234", "420"]:
    mapStr = "map{}-physics.out".format(station)
    if mapStr not in hitmap:
        continue
        
    lossProbability[station] = {}

    length    = []
    occupancy = []
    for point in range(occupancyGraph[station]["integrate"].GetN()):
        xVal = ctypes.c_double(0.0)
        yVal = ctypes.c_double(0.0)
        occupancyGraph[station]["integrate"].GetPoint(point, xVal, yVal)
        
        length    += [xVal.value]
        occupancy += [yVal.value]
        
    lossProbability[station]["length"] = length
    lossProbability[station]["occupancy"] = occupancy
    for timeStep in allTimeSteps:
        probability = lossProbDistrib(timeStep, occupancy)
        
        lossProbability[station]["prob_timeStep{}".format(timeStep)] = probability
        
    for timeStep in drawTimeSteps:
        x, y = array( 'd' ), array( 'd' )
        for ibin in range(len(length)):
            x.append(length[ibin])
            y.append(lossProbability[station]["prob_timeStep{}".format(timeStep)][ibin])
            
        graph = TGraph(len(probability), x, y)
        lossProbability[station]["graph_timeStep{}".format(timeStep)] = graph

In [ ]:
from ROOT import TCanvas, TLegend, TH2D, kAzure

canv = TCanvas("probability_vs_padSize", "Event Loss Probability vs Pad Size", 1200, 1200)
canv.Divide(2,2)

# Create an empty histogram to serve as the frame for the graphs
frame = TH2D("frame", "", 100, 10, 10000, 100, 0.001, 1.1)
frame.SetStats(False)
frame.GetXaxis().SetTitle("Pad Size [#mum]")
frame.GetYaxis().SetTitle("Event Loss Probability")

idx = 0
for station in ["196", "220", "234", "420"]:
    mapStr = "map{}-physics.out".format(station)
    if mapStr not in hitmap:
        continue

    idx += 1
    pad = canv.cd(idx)
    pad.SetLogx()
    pad.SetLogy()
    pad.SetTicks()

    lossProbability[station]["frame"] = frame.Clone("frame-{}".format(station))
    lossProbability[station]["frame"].SetTitle("Station {}".format(station))
    lossProbability[station]["frame"].Draw()
    
    lossProbability[station]["legend"] = TLegend(0.65,0.1,0.9,0.45)
    #lossProbability[station]["legend"].SetHeader("Header", "C")
    
   #legend->AddEntry("g1","0.1 radiation length","l");
   #legend->AddEntry("g2","0.2 radiation length","l");
   #legend->AddEntry("g5","0.5 radiation length","l");
   #legend->AddEntry("g8","0.8 radiation length","l");
   #legend->AddEntry("g15","1.5 radiation length","l");
   #legend->Draw();

    colorOffset = 0
    for timeStep in drawTimeSteps:
        graphName = "graph_timeStep{}".format(timeStep)
        lossProbability[station][graphName].SetLineColor(kAzure + colorOffset)
        lossProbability[station][graphName].Draw("l same")
        lossProbability[station]["legend"].AddEntry(lossProbability[station][graphName], "#tau #leq {} ns".format((timeStep+1)*25), "l")
        colorOffset += 3
        #occupancyGraph[station]["uniform"].Draw("l same")
        #occupancyGraph[station]["integrate"].Draw("l same")
        #occupancyGraph[station]["points"].SetMarkerStyle(20)
        #occupancyGraph[station]["points"].Draw("p same")
    lossProbability[station]["legend"].Draw()

canv.Draw()

In [ ]:
from ROOT import TCanvas, TLegend, TH2D, TH1D, gStyle, kOcean

canv = TCanvas("padSize_vs_deadtime", "Pad Size vs Deadtime", 1200, 1200)
canv.Divide(2,2)

# Create an empty histogram to serve as the frame for the graphs
frame = TH2D("frame", "", 100, 10, 10000, 100, 10, 10000)
#frame = TH1D("frame", "", len(allTimeSteps), 5, 10000)
frame.SetStats(False)
frame.GetXaxis().SetTitle("Deadtime [ns]")
frame.GetYaxis().SetTitle("Pad Size [#mum]")
frame.SetAxisRange(10, 10000,"Y")
frame.SetAxisRange(10, 10000,"X")

probabilities = [1, 2, 5, 10, 20]
probabilities = [20, 10, 5, 2, 1]

idx = 0
for station in ["196", "220", "234", "420"]:
    mapStr = "map{}-physics.out".format(station)
    if mapStr not in hitmap:
        continue

    idx += 1
    pad = canv.cd(idx)
    pad.SetLogx()
    pad.SetLogy()
    pad.SetTicks()

    lossProbability[station]["frame2"] = frame.Clone("frame-{}".format(station))
    lossProbability[station]["frame2"].SetTitle("Station {}".format(station))
    lossProbability[station]["frame2"].Draw()
    
    gStyle.SetPalette(kOcean)
    
    for prob in probabilities:
        hist = TH1D("padSize_vs_deadtime_prob{}_station{}".format(prob, station), "P = {} %".format(prob), len(allTimeSteps), allTimeSteps[0]*25.0 + 0.5, (allTimeSteps[-1]+1)*25.0 + 0.5) # + 0.5 to work around root issue
        length = lossProbability[station]["length"]
        for tIdx in range(len(allTimeSteps)):
            lossProb = lossProbability[station]["prob_timeStep{}".format(allTimeSteps[tIdx])]
            lIdx = 0
            while lossProb[lIdx] < prob/100.0:
                lIdx += 1
            interp = (prob/100.0-lossProb[lIdx-1])/(lossProb[lIdx]-lossProb[lIdx-1])
            probLen = length[lIdx-1] + (length[lIdx] - length[lIdx-1])*interp
            hist.SetBinContent(tIdx+1, probLen)
        hist.Draw("same PLC")
        lossProbability[station]["padVTime_prob{}".format(prob)] = hist
    x = 0.9
    y = 0.9
    pad.BuildLegend(x-0.3, y, x, y-0.21)

canv.Draw()

In [ ]:
def get_shifts(station: str, numShifts:int = 4):
    this_hitmap = hitmap["map{}-physics.out".format(station)]

    # 2e15 neq/cm2 is 4e15 p/cm2
    return this_hitmap.plotShifts(thresholdFlux=2*2e15, integratedLuminosity=300, numCols=3, maxNumShifts=numShifts)

canv, persistance = get_shifts("196")
canv.Draw()

In [ ]:
canv, persistance = get_shifts("220")
canv.Draw()

In [ ]:
canv, persistance = get_shifts("234", numShifts=6)
canv.Draw()

In [ ]:
canv, persistance = get_shifts("420")
canv.Draw()

In [ ]:
import PPSHitmaps

fig = PPSHitmaps.SimpleETLSensor().preview()

In [ ]:
fig = PPSHitmaps.RealisticETLSensor().preview()

In [ ]:
fig = PPSHitmaps.PPSHybrid1Sensor().preview()

In [ ]:
fig = PPSHitmaps.PPSHybrid3Sensor().preview()

In [ ]:
fig = PPSHitmaps.PPSHybrid5Sensor().preview()

In [ ]:
fig = PPSHitmaps.PPSHybrid7Sensor().preview()

In [ ]:
sensorPositions = {
    "196": [
        [(20.61, -15.05)],
    ],
    "220": [
        [(14.86, -10.2)],
        [(14.86, -8.9), (14.86, -11.5)],
    ],
    "234": [
        [(11.86, -3.5)],
        [(11.86, -0.9),(11.86, -3.5),(11.86, -6.1)],
        [(11.86, -2.2),(11.86, -3.5),(11.86, -4.8)],
        [(11.86, -0.9),(11.86, -2.2),(11.86, -3.5),(11.86, -4.8),(11.86, -6.1)],
        [(11.86,  0.4),(11.86, -0.9),(11.86, -2.2),(11.86, -3.5),(11.86, -4.8),(11.86, -6.1),(11.86, -7.4)],
        [(11.86,  1.7),(11.86,  0.4),(11.86, -0.9),(11.86, -2.2),(11.86, -3.5),(11.86, -4.8),(11.86, -6.1),(11.86, -7.4),(11.86, -8.7)],
        [(11.86,  3.0),(11.86,  1.7),(11.86,  0.4),(11.86, -0.9),(11.86, -2.2),(11.86, -3.5),(11.86, -4.8),(11.86, -6.1),(11.86, -7.4),(11.86, -8.7),(11.86, -10.0)],
        [
            (11.86, -1.2 -  0*0.325),
            (11.86, -1.2 -  1*0.325),
            (11.86, -1.2 -  2*0.325),
            (11.86, -1.2 -  3*0.325),
            (11.86, -1.2 -  4*0.325),
            (11.86, -1.2 -  5*0.325),
            (11.86, -1.2 -  6*0.325),
            (11.86, -1.2 -  7*0.325),
            (11.86, -1.2 -  8*0.325),
            (11.86, -1.2 -  9*0.325),
            (11.86, -1.2 - 10*0.325),
            (11.86, -1.2 - 11*0.325),
            (11.86, -1.2 - 12*0.325),
            (11.86, -1.2 - 13*0.325),
            (11.86, -1.2 - 14*0.325),
            (11.86, -1.2 - 15*0.325),
        ],
    ],
    "420": [
        [(17.21,  0.25)],
        [(17.21, -2.35), (17.21,  0.25), (17.21,  2.85)],
    ],
}

In [ ]:
from ROOT import TCanvas
from ROOT import TLine
from ROOT import kRed, kPink, kBlue

import PPSHitmaps

gStyle.SetPalette()

sensor = PPSHitmaps.RealisticETLSensor()
#sensor = PPSHitmaps.PPSHybrid3Sensor()

canv = TCanvas("sensorPosition", "sensorPosition", 2600, 2600)
canv.Divide(2,2)

sensorLines = {}

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    mapStr = "map{}-physics.out".format(station)
    if mapStr not in hitmap:
        continue

    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    #hist_phys[station].SetMinimum(1e6)
    #hist_phys[station].SetMaximum(1e16)
    
    hist_phys[station].Draw("colz")
    
    edge = hitmap["map{}-physics.out".format(station)].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmap["map{}-physics.out".format(station)].yMin * 1000         # Convert to mm for drawing
    yMax = hitmap["map{}-physics.out".format(station)].yMax * 1000         # Convert to mm for drawing
    line[station] = TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(kPink)
    line[station].Draw("same")

    offsetX = hitmap["map{}-physics.out".format(station)].detectorEdge * 1000 - sensor.minX
    for xIdx in hitmap["map{}-physics.out".format(station)].ridge:
        if hitmap["map{}-physics.out".format(station)].ridge[xIdx]['x'] >= hitmap["map{}-physics.out".format(station)].detectorEdge:
            offsetY = hitmap["map{}-physics.out".format(station)].ridge[xIdx]['y']*1000
            break

    offsetX,offsetY = sensorPositions[station][0][0]

    #print(station)
    #print(offsetX)
    #print(offsetY)

    sensorLines[station] = {}
    sensorLines[station]["Left"]   = TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                            sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Right"]  = TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                            sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Top"]    = TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                            sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[station]["Bottom"] = TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                            sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[station]:
        sensorLines[station][key].SetLineColor(kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[station][f"pad{padID}_Left"]   = TLine( minX + offsetX, minY + offsetY,
                                                            minX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Right"]  = TLine( maxX + offsetX, minY + offsetY,
                                                            maxX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Top"]    = TLine( minX + offsetX, minY + offsetY,
                                                            maxX + offsetX, minY + offsetY)
        sensorLines[station][f"pad{padID}_Bottom"] = TLine( minX + offsetX, maxY + offsetY,
                                                            maxX + offsetX, maxY + offsetY)

        sensorLines[station][f"pad{padID}_Left"].SetLineColor(kBlue)
        sensorLines[station][f"pad{padID}_Right"].SetLineColor(kBlue)
        sensorLines[station][f"pad{padID}_Top"].SetLineColor(kBlue)
        sensorLines[station][f"pad{padID}_Bottom"].SetLineColor(kBlue)

        padID += 1

    for key in sensorLines[station]:
        sensorLines[station][key].Draw("same")

canv.Draw()

## ETL-Like LGAD

In [ ]:
import PPSHitmaps

sensor = PPSHitmaps.RealisticETLSensor()
#sensor = PPSHitmaps.PPSHybrid3Sensor()

for station in ["196", "220", "234", "420"]:
    this_hitmap = hitmap["map{}-physics.out".format(station)]
    center_position = sensorPositions[station][0]

    sensor.setShifts(center_position)
    sensor.calculateFlux(this_hitmap)

    print("Station: {}".format(station))
    print("Max Occupancy:")
    print(sensor.findMaxOccupancy())
    print("Dose per fb:")
    print(sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False))
    print()

In [ ]:


#station = "196"
#station = "220"
station = "234"
#station = "420"
this_hitmap = hitmap["map{}-physics.out".format(station)]

center_position = sensorPositions[station][0][0]
print(center_position)

from math import floor
shift_step = 1.3/2
total_shifts = 3
shifts = [(i - floor(total_shifts/2))*shift_step for i in range(total_shifts)]
print(shifts)
position_shifts = [(center_position[0], center_position[1] + shift) for shift in shifts]
print(position_shifts)


sensor.setShifts(position_shifts)
sensor.calculateFlux(this_hitmap)

canv, persistance = sensor.plotLossProbabilityVsDeadtime(timeSteps=5000, minTime=10)

canv.Draw()

In [ ]:
pad_to_plot = 11 # 220
pad_to_plot = 8 # 234

canv, persistance = sensor.padVec[pad_to_plot].plotFlux(usePadSpacing=False)

canv.Draw()

In [ ]:
canv, persistance = sensor.padVec[pad_to_plot].plotDoseEOL(usePadSpacing=False)

canv.Draw()

In [ ]:
from math import floor
shift_step = 1.3/2
total_shifts = 7
shifts = [(i - floor(total_shifts/2))*shift_step for i in range(total_shifts)]
print(shifts)
position_shifts = [(center_position[0], center_position[1] + shift) for shift in shifts]
print(position_shifts)


sensor.setShifts(position_shifts)
sensor.calculateFlux(this_hitmap)

canv, persistance = sensor.padVec[pad_to_plot].plotDoseEOL(usePadSpacing=False)

canv.Draw()

In [ ]:
canv, persistance = sensor.padVec[pad_to_plot].plotDose(canvasHeight = 1600, usePadSpacing = False)

canv.Draw()

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from math import e
from matplotlib import pyplot
from numpy import arange

def chargeFunc(X, Q0, Q1, phi, nu, b, c):
    Volt, Phi = X
    return (Q0 + Q1 * e ** (-Phi/phi)) * e ** (Volt/(nu + b * Phi + c*Phi**2))

chargeData = np.array(
    [
        (150,   0,  4.5), # V, p/cm-2, fC
        (200,   0,  8.5),
        (225,   0, 13),
        (250,   0, 22.5),
        (262.5, 0, 30),
        #-----------------
        (250, 8,  3.5),
        (300, 8,  5),
        (350, 8,  8),
        (375, 8, 11.5),
        (400, 8, 15.5),
        (425, 8, 21),
        (435, 8, 23.5),
        #-----------------
        (450, 15,  4.5),
        (500, 15,  9),
        (525, 15, 11.5),
        (550, 15, 14.5),
        (570, 15, 17),
        #-----------------
        (500, 25,  2),
        (550, 25,  2.5),
        (575, 25,  3),
        (600, 25,  3.5),
        (610, 25,  4),
    ],
    dtype=[("V", "f"), ("Phi", "f"), ("Q", "f")]
)

chargeDF = pd.DataFrame(data=chargeData)


p0=[-0.03, 0.2, 30, 50, 1, -0.01]
popt, pcov = curve_fit(chargeFunc, (chargeDF['V'],chargeDF['Phi']), chargeDF['Q'], p0)

Q0, Q1, phi, nu, b, c = popt

print(popt)

phiVal = 0
x = chargeDF.loc[chargeDF['Phi'] == phiVal]["V"]
y = chargeDF.loc[chargeDF['Phi'] == phiVal]["Q"]
pyplot.scatter(x, y)
x_line = arange(min(x), max(x), 1)
y_line = [chargeFunc((x, phiVal), Q0, Q1, phi, nu, b, c) for x in x_line]
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

phiVal = 8
x = chargeDF.loc[chargeDF['Phi'] == phiVal]["V"]
y = chargeDF.loc[chargeDF['Phi'] == phiVal]["Q"]
pyplot.scatter(x, y)
x_line = arange(min(x), max(x), 1)
y_line = [chargeFunc((x, phiVal), Q0, Q1, phi, nu, b, c) for x in x_line]
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

phiVal = 15
x = chargeDF.loc[chargeDF['Phi'] == phiVal]["V"]
y = chargeDF.loc[chargeDF['Phi'] == phiVal]["Q"]
pyplot.scatter(x, y)
x_line = arange(min(x), max(x), 1)
y_line = [chargeFunc((x, phiVal), Q0, Q1, phi, nu, b, c) for x in x_line]
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

phiVal = 25
x = chargeDF.loc[chargeDF['Phi'] == phiVal]["V"]
y = chargeDF.loc[chargeDF['Phi'] == phiVal]["Q"]
pyplot.scatter(x, y)
x_line = arange(min(x), max(x), 1)
y_line = [chargeFunc((x, phiVal), Q0, Q1, phi, nu, b, c) for x in x_line]
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

Vval = 250
x = chargeDF.loc[chargeDF['V'] == Vval]["Phi"]
y = chargeDF.loc[chargeDF['V'] == Vval]["Q"]
pyplot.scatter(x, y)
x_line = arange(min(x), max(x), 0.1)
y_line = [chargeFunc((Vval, x), Q0, Q1, phi, nu, b, c) for x in x_line]
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

Vval = 500
x = chargeDF.loc[chargeDF['V'] == Vval]["Phi"]
y = chargeDF.loc[chargeDF['V'] == Vval]["Q"]
pyplot.scatter(x, y)
x_line = arange(min(x), max(x), 0.1)
y_line = [chargeFunc((Vval, x), Q0, Q1, phi, nu, b, c) for x in x_line]
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

#chargeDF

In [ ]:
def VoltFunc(Volt, Phi):
    return chargeFunc((Volt, Phi/1e14), Q0, Q1, phi, nu, b, c)

minV, maxV = sensor.padVec[8].getVoltageEOL(chargeFunc=VoltFunc, usePadSpacing=False, integratedLuminosity=210, minCharge=8 )#, chargeDF=chargeDF)

print("Voltage min/max: {} / {}".format(minV, maxV))

In [ ]:
minV, maxV = sensor.padVec[8].getVoltageEOL(chargeFunc=VoltFunc, usePadSpacing=False, integratedLuminosity=300, minCharge=8 )#, chargeDF=chargeDF)

print("Voltage min/max: {} / {}".format(minV, maxV))

In [ ]:
minV, maxV = sensor.padVec[8].getVoltageEOL(chargeFunc=VoltFunc, usePadSpacing=False, integratedLuminosity=100, minCharge=8 )#, chargeDF=chargeDF)

print("Voltage min/max: {} / {}".format(minV, maxV))